# Factor

**Table of contents**

- Overview
- Setup
  - Authentication Token
- Query
  - Output Description
- Related Links

## Overview

This notebook focuses on retrieving specific emission factors from the Factor API. It enables users to:

- Query for emission factors for specific activities and locations
- Override the default factor selection by specifying exact factor sets and versions
- Examine detailed metadata about emission factors, including their source and methodology
- Understand the composition of emission factors across different greenhouse gases

The Factor API gives users precise control over which emissions factors are applied to their calculations. This granular access is essential for regulatory compliance where specific methodologies must be used, for creating transparent audit trails that document exactly which factors were applied, for maintaining consistent reporting methodologies across time periods, and for performing comparative analyses using different factor sets. By exposing the raw emissions factors with their complete metadata, the API supports sophisticated sustainability reporting and compliance workflows.

## Setup


Ensure that you have Python installed in your system. Python 3+ is required.

<b>Note:</b> To run this notebook, you must first add your credentials to `config.read('/../../../auth/secrets.ini')` in the following format:

```
[EAPI]
api.api_key = <Your GHG APIs API key>
api.client_id = <Your GHG APIs client Id>
api.org_id = <Your GHG APIs Org Id>

```

In [ ]:
# Install the packages below using pip/pip3 based on your python version.
%pip install pandas configparser IPython requests

In [17]:
import pandas as pd
import configparser
import requests
import json
from IPython.display import display as display_summary

### Authentication Token



Run the following code snippet to generate the Auth Bearer Token by using your api_key configured in secrets.ini.

In [18]:
config = configparser.RawConfigParser()
config.read(['../../../auth/secrets.ini','../../../auth/config.ini'])

EAPI_API_KEY        = config.get('EAPI', 'api.api_key')
EAPI_TENANT_ID      = config.get('EAPI', 'api.tenant_id')
EAPI_ORG_ID         = config.get('EAPI', 'api.org_id') 

EAPI_AUTH_ENDPOINT  = config.get('EAPI', 'api.auth_endpoint')
EAPI_BASE_URL       = config.get('EAPI', 'api.base_url')
EAPI_ENDPOINT       = f"{EAPI_BASE_URL}/factor"

EAPI_AUTH_CLIENT_ID = 'saascore-' + EAPI_TENANT_ID
EAPI_CLIENT_ID      = 'ghgemissions-' + EAPI_TENANT_ID

auth_request_headers: dict = {}
auth_request_headers["X-IBM-CLIENT-ID"] = EAPI_AUTH_CLIENT_ID
auth_request_headers["X-API-KEY"] = EAPI_API_KEY

verify = True

auth_url = f"{EAPI_AUTH_ENDPOINT}?orgId={EAPI_ORG_ID}"
              
response = requests.get(url = auth_url,
                        headers = auth_request_headers,
                        verify  = verify
                       )
if response.status_code == 200:
    jwt_token = response.text
    print("Authentication Success")
else:     
    print("Authentication Failed")
    print(response.text)

Authentication Success


## Query

The example request payload queries IBM Envizi - Emissions API with location data (India), activity information (electricity consumption measured in kWh), and a specific date (January 1, 2022). By including the optional parameters factorSet: "Managed - NGERS" and factorVersion: "2023", the request explicitly overrides the API's default factor selection algorithm.

In [19]:
payload = {
  "location": {
    "country": "ind"
  },
  "activity": {
    "type": "electricity",
    "unit": "kWh"
  },
  "time": {
    "date": "2022-01-01"
  }
}

In [20]:
# Create the query headers
request_headers: dict = {}
request_headers["Content-Type"] = "application/json"
request_headers["x-ibm-client-id"] = EAPI_CLIENT_ID
request_headers["Authorization"] = "Bearer " + jwt_token

# Submit the request
response = requests.post(EAPI_ENDPOINT, 
                         headers = request_headers, 
                         data = json.dumps(payload))

For more information about allowable parameters for the payload, please see [Emissions API Developer Guide]().

In [21]:
if response.text != "":
    # Get the response as json
    response_json = response.json()
    
    # Get json and convert to dataframe
    json_str = json.dumps(response_json)
    dict = json.loads(json_str)
    dataframe = pd.json_normalize(dict) 
    
    # display
    print("\n\n")
    pd.set_option('display.max_colwidth', None)
    display( dataframe) 
else:
    print("Empty Response")

,transactionId,totalCO2e,unit,description,factorSet,source,activityType,activityUnit,name,effectiveFrom,effectiveTo,publishedFrom,publishedTo,region,factorId
0,844619ac-165d-47cb-9dbc-76ab420d5363,0.960459,kgCO2e,"Grid Emissions Factor- Operating Margin, ex-post incl. imports",GHG - IGES,2025 (v11.6) List of Grid Emissions Factors by Institute for Global Environmental Strategies (IGES). Source: https://www.iges.or.jp/en/publication_documents/pub/data/en/1215/IGES_GRID_EF_v11.6_20250226.xlsx.,Electricity,kWh,"Grid Emissions Factors (GEF) - 2022, ex-post incl. imports",01/01/2022,31/12/2022,01/01/2022,31/12/2022,India,500001950


### Output Description

<b>transactionId</b> - An Emissions API transaction id.

<b>totalCO2e</b>     - The total emissions as CO<sub>2</sub> equivalent (CO<sub>2</sub>e)

<b>CO2</b>           - The amount of CO<sub>2</sub> (Carbon Dioxide) in the CO<sub>2</sub>e value.

<b>CH4</b>           - The amount of CH<sub>4</sub> (Methane) in the CO<sub>2</sub>e value.

<b>N2O</b>           - The amount of N<sub>2</sub>O (Nitrous Oxide) in the CO<sub>2</sub>e value.

<b>HFC</b>           - The amount of HFCs (Hydrofluorocarbons) in the CO<sub>2</sub>e value.

<b>PFC</b>           - The amount of PFCs (Perfluorocarbons) in the CO<sub>2</sub>e value.

<b>SF6</b>           - The amount of SF<sub>6</sub> (Sulphur Hexafluoride) in the CO<sub>2</sub>e value.

<b>NF3</b>           - The amount of NF<sub>3</sub> (Nitrogen Trifluoride) in the CO<sub>2</sub>e value.

<b>bioCo2</b>        - The amount of bio CO<sub>2</sub> in the CO<sub>2</sub> value.

<b>indirectCo2e</b>  - The amount of CO<sub>2</sub>e that is indirect in the CO<sub>2</sub>e value.

<b>unit</b>          - The unit of measure of the values.

<b>description</b>   - A description of the source factor set of the factor used in the calculation.

## Related Links

[Emissions API Developer Guide](https://developer.ibm.com/apis/catalog/ghgemissions--ibm-envizi-emissions-api/Introduction)